# Supporting Documentation & Code

### Initial conversion of CSV to Parquet Files

In [23]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import sys
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import StandardScaler
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoderEstimator, OneHotEncoderModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, MinMaxScaler, MaxAbsScaler, Imputer, VectorAssembler, SQLTransformer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
sc = SparkContext(appName="Final_Project")
sqlContext = SQLContext(sc)

# Add Column ID to testDF

In order to submit to Kaggle we should ensure that the correct id is appended to the dataframe prior to processing with spark as we may lose the ability to know which row id we are evaluating

In [5]:
start = time.time()
testDF = pd.read_csv("test.txt", delimiter="\t", header=None)
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 28.57644557952881 seconds


In [6]:
testDF.insert(0, "id", np.arange(len(testDF)) + 60000000)

In [7]:
testDF = testDF.astype({"id": int})

In [8]:
start = time.time()
testDF.to_csv("test2.txt", sep="\t", index=False, header=False)
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 75.1240165233612 seconds


# Write the CSV's to Parquet

In [9]:
def parseAllColumns(line):
    row = line.split("\t")
    for i in range(14):
        if row[i] == "":
            row[i] = None
        else:
            row[i] = float(row[i])

    return row

In [10]:
# Convert train file from csv to parquet
start = time.time()
schema = StructType([
    StructField("y", FloatType(), True),
    StructField("x1", FloatType(), True),
    StructField("x2", FloatType(), True),
    StructField("x3", FloatType(), True),
    StructField("x4", FloatType(), True),
    StructField("x5", FloatType(), True),
    StructField("x6", FloatType(), True),
    StructField("x7", FloatType(), True),
    StructField("x8", FloatType(), True),
    StructField("x9", FloatType(), True),
    StructField("x10", FloatType(), True),
    StructField("x11", FloatType(), True),
    StructField("x12", FloatType(), True),
    StructField("x13", FloatType(), True),
    StructField("x14", StringType(), True),
    StructField("x15", StringType(), True),
    StructField("x16", StringType(), True),
    StructField("x17", StringType(), True),
    StructField("x18", StringType(), True),
    StructField("x19", StringType(), True),
    StructField("x20", StringType(), True),
    StructField("x21", StringType(), True),
    StructField("x22", StringType(), True),
    StructField("x23", StringType(), True),
    StructField("x24", StringType(), True),
    StructField("x25", StringType(), True),
    StructField("x26", StringType(), True),
    StructField("x27", StringType(), True),
    StructField("x28", StringType(), True),
    StructField("x29", StringType(), True),
    StructField("x30", StringType(), True),
    StructField("x31", StringType(), True),
    StructField("x32", StringType(), True),
    StructField("x33", StringType(), True),
    StructField("x34", StringType(), True),
    StructField("x35", StringType(), True),
    StructField("x36", StringType(), True),
    StructField("x37", StringType(), True),
    StructField("x38", StringType(), True),
    StructField("x39", StringType(), True)])

rdd = sc.textFile('train.txt').map(parseAllColumns)
df = sqlContext.createDataFrame(rdd, schema)
df.write.parquet('train.parquet')
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 450.37015986442566 seconds


In [11]:
# Convert test file from csv to parquet
start = time.time()
schema = StructType([
    StructField("id", DoubleType(), True), # We Created an ID Label
    StructField("x1", FloatType(), True),
    StructField("x2", FloatType(), True),
    StructField("x3", FloatType(), True),
    StructField("x4", FloatType(), True),
    StructField("x5", FloatType(), True),
    StructField("x6", FloatType(), True),
    StructField("x7", FloatType(), True),
    StructField("x8", FloatType(), True),
    StructField("x9", FloatType(), True),
    StructField("x10", FloatType(), True),
    StructField("x11", FloatType(), True),
    StructField("x12", FloatType(), True),
    StructField("x13", FloatType(), True),
    StructField("x14", StringType(), True),
    StructField("x15", StringType(), True),
    StructField("x16", StringType(), True),
    StructField("x17", StringType(), True),
    StructField("x18", StringType(), True),
    StructField("x19", StringType(), True),
    StructField("x20", StringType(), True),
    StructField("x21", StringType(), True),
    StructField("x22", StringType(), True),
    StructField("x23", StringType(), True),
    StructField("x24", StringType(), True),
    StructField("x25", StringType(), True),
    StructField("x26", StringType(), True),
    StructField("x27", StringType(), True),
    StructField("x28", StringType(), True),
    StructField("x29", StringType(), True),
    StructField("x30", StringType(), True),
    StructField("x31", StringType(), True),
    StructField("x32", StringType(), True),
    StructField("x33", StringType(), True),
    StructField("x34", StringType(), True),
    StructField("x35", StringType(), True),
    StructField("x36", StringType(), True),
    StructField("x37", StringType(), True),
    StructField("x38", StringType(), True),
    StructField("x39", StringType(), True)])

rdd = sc.textFile('test2.txt').map(parseAllColumns)
df = sqlContext.createDataFrame(rdd, schema)
df.write.parquet('test.parquet')
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 63.724459171295166 seconds


In [5]:
# start Spark Session
app_name = "final_project_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()

In [6]:
spark

In [7]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [8]:
sc._conf.getAll()

[('spark.driver.memory', '25g'),
 ('spark.driver.port', '38519'),
 ('spark.app.name', 'final_project_notebook'),
 ('spark.app.id', 'local-1574524168976'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', 'docker.w261'),
 ('spark.executor.memory', '25g')]

# Create Toy Example

In [13]:
testDF = sqlContext.read.parquet('test.parquet') # This loads a Data Frame
trainDF = sqlContext.read.parquet('train.parquet')

In [14]:
HEADER = trainDF.columns
INTEGER_FEATURES = HEADER[0:14] # These are the integer features
CATEGORICAL_FEATURES = HEADER[14:] # These are the categorical features

In [15]:
sample = trainDF.sample(False, 0.0001, seed=1234).toPandas() # Approximately 5000 records

In [22]:
toy_example = sample[5:15]
toy_example.reset_index(inplace=True,drop=True)

In [23]:
small_toy_example = toy_example[["y","x1","x2","x6","x8","x14","x19","x20","x35"]]

In [24]:
#small_toy_example.to_csv("toy_example.txt",index=None)
small_toy_example

,y,x1,x2,x6,x8,x14,x19,x20,x35
0,1.0,2.0,671.0,145.0,12.0,05db9164,fbad5c96,6c5e14ec,
1,0.0,0.0,-1.0,9.0,0.0,05db9164,6f6d9be8,2f5788d6,
2,0.0,NaN,0.0,100.0,0.0,8cf07265,7e0ccccf,2cc59e2b,ad3062eb
3,0.0,NaN,-1.0,NaN,0.0,05db9164,fbad5c96,d356c7e6,
4,0.0,NaN,1.0,203.0,5.0,68fd1e64,fbad5c96,d5f62b87,
5,0.0,1.0,-1.0,0.0,0.0,05db9164,,1b76cf1e,
6,1.0,NaN,39.0,NaN,117.0,5bfa8ab5,7e0ccccf,af0809a5,
7,0.0,NaN,0.0,66.0,7.0,05db9164,,da33ebe6,
8,1.0,10.0,1.0,66.0,27.0,05db9164,fbad5c96,ce4f7f55,
9,0.0,NaN,1.0,16.0,7.0,68fd1e64,7e0ccccf,5e64ce5f,


# K-Fold Cross Validation Grid Search for Regularization and Elastic Net Parameter

# Submission of Data to Kaggle Our Test Set to Kaggle To See Performance

In [55]:
kaggleOutput = testPredsAndLabels.select(["id", "probability"])
kaggleOutput = kaggleOutput.rdd.map(lambda x: (int(x['id']), float(x['probability'][1])))

In [169]:
# Test Some Brief Stuff

kaggleOutput = lr_test_predictions.select(["id", "probability"])
kaggleOutput = kaggleOutput.rdd.map(lambda x: (int(x['id']), float(x['probability'][1])))

In [170]:
# Convert train file from csv to parquet
start = time.time()
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("probability", FloatType(), True)])

kaggleDF = sqlContext.createDataFrame(kaggleOutput, schema)
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 0.05407404899597168 seconds


In [171]:
start = time.time()
kaggleDF.write.parquet('kaggle.parquet')
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 42.62530779838562 seconds


In [172]:
start = time.time()
kaggleDF = sqlContext.read.parquet('kaggle.parquet')
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 0.035701751708984375 seconds


In [173]:
start = time.time()
pandasDF = kaggleDF.toPandas()
pandasDF.columns=['id','Predicted']
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 14.604223251342773 seconds


In [174]:
start = time.time()
pandasDF.to_csv('submission.csv', header=True, index=False)
print(f"\n... Executed in {time.time() - start} seconds")


... Executed in 11.946702718734741 seconds


In [175]:
# !kaggle competitions list

In [176]:
!kaggle competitions submit -c criteo-display-ad-challenge -f submission.csv -m "Baseline Submission"

100%|████████████████████████████████████████| 114M/114M [01:40<00:00, 1.19MB/s]
Successfully submitted to Display Advertising Challenge